In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

In [2]:
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config.get('AWS', 'AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY']=config.get('AWS', 'AWS_SECRET_ACCESS_KEY')

In [3]:

def create_spark_session():
    ''' Create a Spark Session and return the spark session object'''
    
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

In [37]:
#spark = create_spark_session()
input_data = "s3a://udacity-dend/"
output_data = "s3a://praveen-practice-bucket/"
#output_data = "s3a://udacity-dend/"

In [5]:
print(spark)

In [19]:
# get filepath to song data file
song_data = os.path.join(input_data,'song_data/A/A/B/*.json')


In [26]:
# read song data file
df = spark.read.json(song_data)

In [27]:
df.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: double (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: double (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [28]:
df.show(3)

+------------------+---------------+--------------------+----------------+---------------+---------+---------+------------------+--------------------+----+
|         artist_id|artist_latitude|     artist_location|artist_longitude|    artist_name| duration|num_songs|           song_id|               title|year|
+------------------+---------------+--------------------+----------------+---------------+---------+---------+------------------+--------------------+----+
|ARKIQCZ1187B9A7C7C|       52.23974|Northampton, Nort...|        -0.88576|        Bauhaus|248.65914|        1|SOSIJKW12A8C1330E3|A God In An Alcov...|   0|
|ARI9DQS1187B995625|       44.93746|  Chippewa Falls, WI|       -91.39251|    Judy Henske|139.78077|        1|SODVVEL12A6D4F9EA0|Good Old Wagon (L...|1964|
|AR2L9A61187B9ADDBC|       52.51607|     Berlin, Germany|        13.37698|Tangerine Dream|492.30322|        1|SOKPKMV12A8C14125E|Catwalk (Black In...|1995|
+------------------+---------------+--------------------+-------

In [31]:
# extract columns to create songs table
songs_table = df.select("song_id", "title", "artist_id", "duration", "year").dropDuplicates()
print('exrtacted data for songs_table')
songs_table.show(2)

exrtacted data for songs_table
+------------------+--------------------+------------------+---------+----+
|           song_id|               title|         artist_id| duration|year|
+------------------+--------------------+------------------+---------+----+
|SOKPKMV12A8C14125E|Catwalk (Black In...|AR2L9A61187B9ADDBC|492.30322|1995|
|SOSRVUJ12AB018731E|       Walk the Walk|ARD8E0V1187FB5C3DB|290.16771|2000|
+------------------+--------------------+------------------+---------+----+
only showing top 2 rows



In [38]:
# write songs table to parquet files partitioned by year and artist
songs_table = songs_table.write.partitionBy(['year', 'artist_id']).parquet(os.path.join(output_data, "songs_table_yo.parquet"))
print('saved songs_table')

# extract columns to create artists table
#artists_table = df.select(['artist_id', 'artist_name', 'artist_location', 'artist_lattitude', 'artist_longitude']).dropDuplicates()

Py4JJavaError: An error occurred while calling o321.parquet.
: com.amazonaws.services.s3.model.AmazonS3Exception: Status Code: 400, AWS Service: Amazon S3, AWS Request ID: 6D3E3B8FCBA4457C, AWS Error Code: null, AWS Error Message: Bad Request, S3 Extended Request ID: N5khAimwxNY/kjhiYsqNqRKpBPNruxXKaroe2iEczZHpZn+YNf4WZNUJpstDsKWL4edjuFG7b7s=
	at com.amazonaws.http.AmazonHttpClient.handleErrorResponse(AmazonHttpClient.java:798)
	at com.amazonaws.http.AmazonHttpClient.executeHelper(AmazonHttpClient.java:421)
	at com.amazonaws.http.AmazonHttpClient.execute(AmazonHttpClient.java:232)
	at com.amazonaws.services.s3.AmazonS3Client.invoke(AmazonS3Client.java:3528)
	at com.amazonaws.services.s3.AmazonS3Client.headBucket(AmazonS3Client.java:1031)
	at com.amazonaws.services.s3.AmazonS3Client.doesBucketExist(AmazonS3Client.java:994)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.initialize(S3AFileSystem.java:297)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2669)
	at org.apach